# Data acquisition 
What is done:
- get connection url to zillow data set in sql
- create sql query that meets the requirements
- using the sql query create a zillow database
- encode that database to a csv file

##### Project Requirements:
- ML regression model that predicts `taxvaluedollarcnt` of **Single Family Properties**
- Use properties that had a transaction in 2017
- Tables: 
    - properties_2017
    - predictions_2017
    - propertylandusetype
- Features sudjestions to use:
    - SQFT
    - number of bedrooms
    - number of bathrooms
    - feature engineer new coloumn: number of rooms
- Create a table that tells the zillow data team which state and county the houses are located in (fips)

In [1]:
# imports:
import pandas as pd
import numpy as np

#personal
import env 
import os
import wrangle as wr

In [2]:
# function to get connectio url: 
def get_db_url(db, user= env.user, host=env.host, password=env.password):
    """
    This function will:
    - take credentials from env.py file
    - make a connection to the SQL database with given credentials
    - return url connection
    """
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

In [3]:
# need to get connection url 
url = get_db_url('zillow')

In [4]:
# create a zillow query (I know that this runs in sql)
sql_query_2 = '''
SELECT 
    id,
    transactiondate,
    bathroomcnt,
    bedroomcnt,
    calculatedfinishedsquarefeet,
    fips,
    taxvaluedollarcnt,
    propertylandusetypeid,
    propertylandusedesc,
    yearbuilt
FROM 
    predictions_2017
LEFT JOIN properties_2017
    USING (id)
LEFT JOIN propertylandusetype
    USING (propertylandusetypeid)
WHERE propertylandusetypeid = 261
and transactiondate LIKE '2017%%'
and propertylandusedesc='Single Family Residential'

'''

In [5]:
df_2 = pd.read_sql(sql_query_2, url)
df_2.head()

,id,transactiondate,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,taxvaluedollarcnt,propertylandusetypeid,propertylandusedesc,yearbuilt
0,3747,2017-01-18,2.0,3.0,1524.0,6037.0,336256.0,261.0,Single Family Residential,1948.0
1,3746,2017-01-18,2.0,5.0,1562.0,6037.0,492419.0,261.0,Single Family Residential,1956.0
2,3745,2017-01-18,2.0,3.0,1304.0,6037.0,88594.0,261.0,Single Family Residential,1953.0
3,3744,2017-01-18,2.0,3.0,1540.0,6037.0,100205.0,261.0,Single Family Residential,1956.0
4,3743,2017-01-18,2.0,3.0,1172.0,6037.0,536000.0,261.0,Single Family Residential,1948.0


In [6]:
df_2.transactiondate.max()

'2017-09-21'

In [7]:
df_2.describe()

,id,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,taxvaluedollarcnt,propertylandusetypeid,yearbuilt
count,56079.000000,56079.000000,56079.000000,55848.000000,56079.000000,5.607100e+04,56079.0,55830.000000
mean,38683.158419,2.239457,3.289110,1868.285704,6048.841581,4.576434e+05,261.0,1961.186262
std,22360.841618,1.007012,0.958756,966.973076,21.237610,6.635581e+05,0.0,21.995171
min,1.000000,0.000000,0.000000,20.000000,6037.000000,9.000000e+00,261.0,1862.000000
25%,19467.000000,2.000000,3.000000,1260.000000,6037.000000,1.870260e+05,261.0,1949.000000
50%,38805.000000,2.000000,3.000000,1630.000000,6037.000000,3.295250e+05,261.0,1958.000000
75%,57163.500000,3.000000,4.000000,2217.000000,6059.000000,5.334845e+05,261.0,1976.000000
max,77611.000000,20.000000,25.000000,26345.000000,6111.000000,6.750649e+07,261.0,2016.000000


In [8]:
# a better sql query
sql_query = '''
SELECT
    transactiondate,
    bathroomcnt,
    bedroomcnt,
    calculatedfinishedsquarefeet,
    fips,
    taxvaluedollarcnt,
    propertylandusetypeid,
    propertylandusedesc,
    yearbuilt
FROM properties_2017
    JOIN predictions_2017 USING(parcelid)
    JOIN propertylandusetype USING(propertylandusetypeid)
    WHERE propertylandusedesc='Single Family Residential' AND transactiondate LIKE '2017%%';'''

In [9]:
# acquire the zillow data
df = pd.read_sql(sql_query, url)
df.head()

,transactiondate,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,taxvaluedollarcnt,propertylandusetypeid,propertylandusedesc,yearbuilt
0,2017-01-13,2.0,3.0,1324.0,6037.0,518089.0,261.0,Single Family Residential,1963.0
1,2017-01-13,3.0,4.0,3551.0,6037.0,1609238.0,261.0,Single Family Residential,1994.0
2,2017-01-13,3.0,4.0,2519.0,6037.0,106406.0,261.0,Single Family Residential,1965.0
3,2017-01-13,2.0,3.0,1285.0,6037.0,369853.0,261.0,Single Family Residential,1955.0
4,2017-01-13,1.0,3.0,1254.0,6037.0,65848.0,261.0,Single Family Residential,1954.0


In [10]:
df.shape

(52441, 9)

In [11]:
df.transactiondate.max()

'2017-09-25'

In [12]:
df.propertylandusetypeid.value_counts()

261.0    52441
Name: propertylandusetypeid, dtype: int64

In [13]:
df.propertylandusedesc.value_counts()

Single Family Residential    52441
Name: propertylandusedesc, dtype: int64

In [14]:
# make a function based on this data:
def new_zillow_data():
    '''
    This function will:
    - read a set sql query
    - return a dataframe based on the given query
    '''

    zillow_query = '''
    SELECT
        transactiondate,
        bathroomcnt,
        bedroomcnt,
        calculatedfinishedsquarefeet,
        fips,
        taxvaluedollarcnt,
        propertylandusetypeid,
        propertylandusedesc,
        yearbuilt
    FROM properties_2017
        JOIN predictions_2017 USING(parcelid)
        JOIN propertylandusetype USING(propertylandusetypeid)
    WHERE propertylandusedesc='Single Family Residential' AND transactiondate LIKE '2017%%';
        '''
        
    # read in the dataframe from codeup
    df = pd.read_sql(zillow_query, get_db_url('zillow'))
    
    return df

In [15]:
df = new_zillow_data()
df.head()

,transactiondate,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,taxvaluedollarcnt,propertylandusetypeid,propertylandusedesc,yearbuilt
0,2017-01-13,2.0,3.0,1324.0,6037.0,518089.0,261.0,Single Family Residential,1963.0
1,2017-01-13,3.0,4.0,3551.0,6037.0,1609238.0,261.0,Single Family Residential,1994.0
2,2017-01-13,3.0,4.0,2519.0,6037.0,106406.0,261.0,Single Family Residential,1965.0
3,2017-01-13,2.0,3.0,1285.0,6037.0,369853.0,261.0,Single Family Residential,1955.0
4,2017-01-13,1.0,3.0,1254.0,6037.0,65848.0,261.0,Single Family Residential,1954.0


In [16]:
# make the data set into a csv
def get_zillow_data():
    '''
    This functino will check for a zillow.csv,
    If it exists it will pull data from said file.
    '''
    
    if os.path.isfile('zillow.csv'):
        #if csv file exists read in data from csv file:
        df = pd.read_csv('zillow.csv', index_col = 0)
        
    else:
        
        #read the fresh data form db into a dataframe
        df = new_zillow_data()
        
        #cache data:
        df.to_csv('zillow.csv')
    
    return df

In [17]:
df = get_zillow_data()
df.head()

,transactiondate,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,fips,taxvaluedollarcnt,propertylandusetypeid,propertylandusedesc,yearbuilt
0,2017-01-13,2.0,3.0,1324.0,6037.0,518089.0,261.0,Single Family Residential,1963.0
1,2017-01-13,3.0,4.0,3551.0,6037.0,1609238.0,261.0,Single Family Residential,1994.0
2,2017-01-13,3.0,4.0,2519.0,6037.0,106406.0,261.0,Single Family Residential,1965.0
3,2017-01-13,2.0,3.0,1285.0,6037.0,369853.0,261.0,Single Family Residential,1955.0
4,2017-01-13,1.0,3.0,1254.0,6037.0,65848.0,261.0,Single Family Residential,1954.0


### Initial Questions from looking at the dataset
Could these features be factors? 
- `yearbuilt` : How old the house is?
- `lotsizesqaurefeet` : How big is the property?
- `numberofstories` : Does number of stories influence purchase? (need to remove outliers to make data normal)

Features that were looked into but don't look reliable(outliers)
- `fullbathcnt` : This is the same as bathroomcnt
- `roomcnt` : How many rooms are there, is this even actuarte? (Not reliable)